In [1]:
# https://catboost.ai/en/docs/concepts/python-quickstart
import catboost as cb
from catboost import CatBoostClassifier


print ('catboost version', cb.__version__)

catboost version 1.0.6


# K-Fold CV for Boosting Methods Using SKLearn 

In [2]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import wfdb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
import random
import statistics

In [3]:
import xgboost as xgb
from sklearn import metrics
from lightgbm import LGBMClassifier

import lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats

### To load sklearn dataset
sklearn.datasets.load_breast_cancer(*, return_X_y=False, as_frame=False)[source]¶

Load and return the breast cancer wisconsin dataset (classification).

In [5]:
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

        

  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [00:22<00:00,  1.14it/s]


In [6]:
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
#features_maindf = pd.concat(features_df_list, ignore_index = True)

In [7]:
features_maindf = pd.concat(features_df_list, ignore_index = True)
print(features_maindf.columns)

Index(['Unnamed: 0', 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation',
       'Rhythm', 'Subject'],
      dtype='object')


# CatBoosting

In [14]:
def confusion_calcs(confusion_matrix):
    temp_arr = confusion_matrix
    tn = temp_arr[0]
    fp = temp_arr[1]
    fn = temp_arr[2]
    tp = temp_arr[3]
    sensitivity = tp / (tp + fn)
    recall = sensitivity
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    accuracy = (tp+tn) / (tp+tn+fn+fp)
    f1 = 2*precision*recall/(precision+recall)
    return [accuracy, sensitivity, specificity, precision, f1]

In [15]:
#Ignore the Rest of the STuff. Main One

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

#acc_score = [];
#Truth = [];
#Output = [];
iterations_list = []
#max_depth_list = []
#x_values = ['StoS']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf



all_features = ['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
        'AppEn', 'SampEn']
dataScores = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        
for score_idx,temp in tqdm(enumerate(all_features)):    
    
    x_values = [temp]
    print(x_values)

    acc_score = [];
    Truth = [];
    Output = [];



    for idx, subject in enumerate(good_list):

        train_index = ~df['Subject'].str.contains(subject)
        test_index = df['Subject'].str.contains(subject)
        X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
        y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()

        model.fit(X_train, y_train)

        # make the prediction using the resulting model
        pred_values = model.predict(X_test)
        #preds_proba = model.predict_proba(test_data)
        #print("class = ", preds_class)
        #print("proba = ", preds_proba)

        acc = accuracy_score(y_test, pred_values)
        acc_score.append(acc)

        Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
        Output.extend(pred_values); ## it is a list  

        #print(model.feature_importances_)
    #     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
    #                           index = (X_train.columns))
    #     Importance.sort_values(by = 'Importance', 
    #                        axis = 0, 
    #                        ascending = True).plot(kind = 'barh', 
    #                                               color = 'r')
    #     plt.xlabel('Variable Importance')
    #     plt.gca().legend_ = None
        #plt.savefig('plot1.png')


    elapsed = timeit.default_timer() - start_time #gives total computation time
    print("---Run time is %s seconds ---" % elapsed) #prints computation time
    print()

    print('Accuracy of each fold: \n {}'.format(acc_score))
    print()
    try: 
        print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
    except: 
        print('hi')
    print()
    print('Std of accuracy : \n{}'.format(np.std(acc_score)))
    print()
    print(confusion_matrix(Truth, Output))
    print()
    print(classification_report(Truth, Output))
    print()
    print(acc_score)
    print()
    print(elapsed)
    
    temp_array = confusion_matrix(Truth, Output)
    joint = temp_array[0].tolist() + temp_array[1].tolist()
    temp_scores = confusion_calcs(joint)
    
    #dataScores[score_idx].append(all_features[score_idx])
    #dataScores[score_idx].append(acc_score)
    dataScores[score_idx].append(np.mean(acc_score))
    dataScores[score_idx].append(np.std(acc_score))
    
    for confusionScore in temp_scores:
        dataScores[score_idx].append(confusionScore)

0it [00:00, ?it/s]

['StoS']
0:	learn: 0.6656310	total: 157ms	remaining: 15.5s
1:	learn: 0.6431669	total: 160ms	remaining: 7.84s
2:	learn: 0.6245059	total: 163ms	remaining: 5.28s
3:	learn: 0.6094615	total: 166ms	remaining: 3.99s
4:	learn: 0.5961791	total: 170ms	remaining: 3.22s
5:	learn: 0.5856859	total: 172ms	remaining: 2.69s
6:	learn: 0.5766557	total: 175ms	remaining: 2.32s
7:	learn: 0.5686885	total: 177ms	remaining: 2.04s
8:	learn: 0.5623394	total: 180ms	remaining: 1.82s
9:	learn: 0.5571202	total: 182ms	remaining: 1.64s
10:	learn: 0.5526467	total: 184ms	remaining: 1.49s
11:	learn: 0.5487583	total: 187ms	remaining: 1.37s
12:	learn: 0.5454487	total: 190ms	remaining: 1.27s
13:	learn: 0.5425911	total: 192ms	remaining: 1.18s
14:	learn: 0.5402381	total: 195ms	remaining: 1.1s
15:	learn: 0.5385129	total: 197ms	remaining: 1.03s
16:	learn: 0.5368412	total: 200ms	remaining: 974ms
17:	learn: 0.5354000	total: 202ms	remaining: 920ms
18:	learn: 0.5341129	total: 205ms	remaining: 873ms
19:	learn: 0.5331089	total: 207ms

92:	learn: 0.4993895	total: 227ms	remaining: 17.1ms
93:	learn: 0.4993906	total: 230ms	remaining: 14.7ms
94:	learn: 0.4993875	total: 232ms	remaining: 12.2ms
95:	learn: 0.4993889	total: 234ms	remaining: 9.74ms
96:	learn: 0.4993880	total: 236ms	remaining: 7.29ms
97:	learn: 0.4993850	total: 238ms	remaining: 4.86ms
98:	learn: 0.4993850	total: 240ms	remaining: 2.43ms
99:	learn: 0.4993866	total: 243ms	remaining: 0us
0:	learn: 0.6660973	total: 2.76ms	remaining: 273ms
1:	learn: 0.6440193	total: 5.3ms	remaining: 260ms
2:	learn: 0.6256533	total: 7.96ms	remaining: 257ms
3:	learn: 0.6108796	total: 10.1ms	remaining: 242ms
4:	learn: 0.5978085	total: 13.2ms	remaining: 250ms
5:	learn: 0.5875006	total: 15.5ms	remaining: 243ms
6:	learn: 0.5786065	total: 18.1ms	remaining: 240ms
7:	learn: 0.5707749	total: 20.8ms	remaining: 239ms
8:	learn: 0.5645434	total: 23.4ms	remaining: 237ms
9:	learn: 0.5594236	total: 25.9ms	remaining: 234ms
10:	learn: 0.5550077	total: 28.6ms	remaining: 231ms
11:	learn: 0.5515128	total

69:	learn: 0.5267106	total: 171ms	remaining: 73.1ms
70:	learn: 0.5267061	total: 173ms	remaining: 70.6ms
71:	learn: 0.5267038	total: 175ms	remaining: 68.1ms
72:	learn: 0.5267029	total: 177ms	remaining: 65.4ms
73:	learn: 0.5267019	total: 179ms	remaining: 62.8ms
74:	learn: 0.5266995	total: 181ms	remaining: 60.4ms
75:	learn: 0.5266967	total: 184ms	remaining: 58.1ms
76:	learn: 0.5266937	total: 186ms	remaining: 55.7ms
77:	learn: 0.5266929	total: 189ms	remaining: 53.3ms
78:	learn: 0.5266902	total: 192ms	remaining: 50.9ms
79:	learn: 0.5266838	total: 195ms	remaining: 48.7ms
80:	learn: 0.5266878	total: 197ms	remaining: 46.2ms
81:	learn: 0.5266834	total: 199ms	remaining: 43.7ms
82:	learn: 0.5266796	total: 202ms	remaining: 41.3ms
83:	learn: 0.5266762	total: 204ms	remaining: 38.9ms
84:	learn: 0.5266754	total: 207ms	remaining: 36.5ms
85:	learn: 0.5266718	total: 209ms	remaining: 34ms
86:	learn: 0.5266749	total: 211ms	remaining: 31.6ms
87:	learn: 0.5266709	total: 214ms	remaining: 29.1ms
88:	learn: 0.5

33:	learn: 0.4913187	total: 87.7ms	remaining: 170ms
34:	learn: 0.4912237	total: 90.2ms	remaining: 168ms
35:	learn: 0.4911338	total: 92.6ms	remaining: 165ms
36:	learn: 0.4910824	total: 94.5ms	remaining: 161ms
37:	learn: 0.4910214	total: 97.2ms	remaining: 159ms
38:	learn: 0.4909506	total: 99.8ms	remaining: 156ms
39:	learn: 0.4908840	total: 102ms	remaining: 154ms
40:	learn: 0.4908327	total: 105ms	remaining: 151ms
41:	learn: 0.4908028	total: 107ms	remaining: 148ms
42:	learn: 0.4907716	total: 110ms	remaining: 146ms
43:	learn: 0.4907516	total: 113ms	remaining: 144ms
44:	learn: 0.4907177	total: 115ms	remaining: 141ms
45:	learn: 0.4907044	total: 118ms	remaining: 138ms
46:	learn: 0.4906954	total: 120ms	remaining: 135ms
47:	learn: 0.4906917	total: 122ms	remaining: 132ms
48:	learn: 0.4906703	total: 124ms	remaining: 129ms
49:	learn: 0.4906446	total: 127ms	remaining: 127ms
50:	learn: 0.4906242	total: 130ms	remaining: 125ms
51:	learn: 0.4906158	total: 132ms	remaining: 122ms
52:	learn: 0.4906022	tota

...

50:	learn: 0.2400327	total: 171ms	remaining: 164ms
51:	learn: 0.2398130	total: 174ms	remaining: 161ms
52:	learn: 0.2396522	total: 177ms	remaining: 157ms
53:	learn: 0.2395803	total: 181ms	remaining: 154ms
54:	learn: 0.2394481	total: 184ms	remaining: 150ms
55:	learn: 0.2393998	total: 187ms	remaining: 147ms
56:	learn: 0.2393155	total: 190ms	remaining: 144ms
57:	learn: 0.2392391	total: 194ms	remaining: 140ms
58:	learn: 0.2392121	total: 197ms	remaining: 137ms
59:	learn: 0.2392037	total: 201ms	remaining: 134ms
60:	learn: 0.2391497	total: 204ms	remaining: 130ms
61:	learn: 0.2390140	total: 207ms	remaining: 127ms
62:	learn: 0.2389654	total: 211ms	remaining: 124ms
63:	learn: 0.2389440	total: 215ms	remaining: 121ms
64:	learn: 0.2388919	total: 218ms	remaining: 117ms
65:	learn: 0.2388442	total: 221ms	remaining: 114ms
66:	learn: 0.2388139	total: 224ms	remaining: 110ms
67:	learn: 0.2387499	total: 227ms	remaining: 107ms
68:	learn: 0.2386324	total: 231ms	remaining: 104ms
69:	learn: 0.2385977	total: 234

56:	learn: 0.2353926	total: 182ms	remaining: 137ms
57:	learn: 0.2353013	total: 186ms	remaining: 134ms
58:	learn: 0.2352693	total: 189ms	remaining: 131ms
59:	learn: 0.2352361	total: 192ms	remaining: 128ms
60:	learn: 0.2351847	total: 195ms	remaining: 125ms
61:	learn: 0.2351326	total: 199ms	remaining: 122ms
62:	learn: 0.2350888	total: 202ms	remaining: 119ms
63:	learn: 0.2350223	total: 205ms	remaining: 116ms
64:	learn: 0.2349737	total: 209ms	remaining: 112ms
65:	learn: 0.2349538	total: 212ms	remaining: 109ms
66:	learn: 0.2348636	total: 215ms	remaining: 106ms
67:	learn: 0.2347831	total: 219ms	remaining: 103ms
68:	learn: 0.2347196	total: 222ms	remaining: 99.7ms
69:	learn: 0.2346892	total: 225ms	remaining: 96.4ms
70:	learn: 0.2346495	total: 229ms	remaining: 93.4ms
71:	learn: 0.2346144	total: 232ms	remaining: 90.1ms
72:	learn: 0.2346004	total: 235ms	remaining: 86.9ms
73:	learn: 0.2345267	total: 238ms	remaining: 83.6ms
74:	learn: 0.2344944	total: 241ms	remaining: 80.4ms
75:	learn: 0.2344683	tot

56:	learn: 0.2338569	total: 182ms	remaining: 137ms
57:	learn: 0.2337346	total: 186ms	remaining: 134ms
58:	learn: 0.2336246	total: 189ms	remaining: 131ms
59:	learn: 0.2336070	total: 192ms	remaining: 128ms
60:	learn: 0.2335546	total: 195ms	remaining: 125ms
61:	learn: 0.2334975	total: 199ms	remaining: 122ms
62:	learn: 0.2334423	total: 203ms	remaining: 119ms
63:	learn: 0.2333549	total: 206ms	remaining: 116ms
64:	learn: 0.2333131	total: 210ms	remaining: 113ms
65:	learn: 0.2333006	total: 213ms	remaining: 110ms
66:	learn: 0.2332715	total: 216ms	remaining: 107ms
67:	learn: 0.2331966	total: 220ms	remaining: 103ms
68:	learn: 0.2331627	total: 223ms	remaining: 100ms
69:	learn: 0.2331212	total: 227ms	remaining: 97.3ms
70:	learn: 0.2330894	total: 230ms	remaining: 94ms
71:	learn: 0.2330212	total: 234ms	remaining: 90.9ms
72:	learn: 0.2330131	total: 237ms	remaining: 87.7ms
73:	learn: 0.2329319	total: 240ms	remaining: 84.5ms
74:	learn: 0.2328660	total: 244ms	remaining: 81.3ms
75:	learn: 0.2328014	total:

16it [02:16,  8.54s/it]

              precision    recall  f1-score   support

           0       0.91      0.85      0.88     24743
           1       0.83      0.89      0.86     20225

    accuracy                           0.87     44968
   macro avg       0.87      0.87      0.87     44968
weighted avg       0.87      0.87      0.87     44968


[0.8832951945080092, 0.9460446247464503, 0.9924433249370277, 0.6241007194244604, 0.9822361546499477, 0.9245436105476673, 0.9056515646893974, 0.988427501701838, 0.9292168674698795, 0.7938257993384785, 0.9818511796733213, 0.9547738693467337, 0.6970521541950113, 1.0, 0.35428809325562033, 0.9060700044306602, 0.9677861549006168, 0.9982688978649741, 0.5841081994928149, 0.9516217702034084, 0.9798973481608212, 0.9133709981167608, 0.9674418604651163]

136.54770630000007


In [16]:
    
export_df = pd.DataFrame(dataScores,columns=['Mean Acc.', 'SD of Acc.', 'Total Acc.', 'Sensitivity', 
                                             'Specificity','Precision','f1'])

export_df = export_df.set_index(pd.Index(['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
        'AppEn', 'SampEn']))

In [17]:
export_df

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898461,0.112915,0.891367,0.934289,0.856283,0.841618,0.885536
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845425,0.132364,0.836328,0.841879,0.831791,0.803577,0.822282
RtoR,0.884048,0.139409,0.875600,0.902596,0.853534,0.834362,0.867139
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785249,0.178835,0.769325,0.542744,0.954533,0.907040,0.679123
LtoR,0.822828,0.118022,0.820428,0.856663,0.790810,0.769976,0.811009
LtoL,0.692266,0.243358,0.670610,0.306848,0.967951,0.886698,0.455921
STD,0.784341,0.197660,0.782668,0.868381,0.712606,0.711802,0.782334


In [18]:
export_df_highlight = export_df.copy()

export_df_highlight.style.highlight_max(color = 'lightgreen', axis = 0)

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898461,0.112915,0.891367,0.934289,0.856283,0.841618,0.885536
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845425,0.132364,0.836328,0.841879,0.831791,0.803577,0.822282
RtoR,0.884048,0.139409,0.875600,0.902596,0.853534,0.834362,0.867139
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785249,0.178835,0.769325,0.542744,0.954533,0.907040,0.679123
LtoR,0.822828,0.118022,0.820428,0.856663,0.790810,0.769976,0.811009
LtoL,0.692266,0.243358,0.670610,0.306848,0.967951,0.886698,0.455921
STD,0.784341,0.197660,0.782668,0.868381,0.712606,0.711802,0.782334


In [19]:
export_df_highlight.style.highlight_min(color = 'red', axis = 0)

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898461,0.112915,0.891367,0.934289,0.856283,0.841618,0.885536
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845425,0.132364,0.836328,0.841879,0.831791,0.803577,0.822282
RtoR,0.884048,0.139409,0.875600,0.902596,0.853534,0.834362,0.867139
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785249,0.178835,0.769325,0.542744,0.954533,0.907040,0.679123
LtoR,0.822828,0.118022,0.820428,0.856663,0.790810,0.769976,0.811009
LtoL,0.692266,0.243358,0.670610,0.306848,0.967951,0.886698,0.455921
STD,0.784341,0.197660,0.782668,0.868381,0.712606,0.711802,0.782334


In [20]:
export_df.to_csv(f'individual-features/CatBoost.csv')

In [8]:
#Ignore Stuff Below

In [9]:
#Ignore Stuff Below

In [10]:
#Ignore Stuff Below

In [11]:
#Ignore Stuff Below

In [12]:
#Ignore Stuff Below

In [13]:
#Ignore Stuff Below